In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator, RegressorMixin

In [2]:
# data loading
boston=load_boston()
print(boston.data.shape)

(506, 13)


In [3]:
# checking feature
print(boston.feature_names)

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']


In [4]:
# setting X and y
mean=np.mean(boston.data, axis=0)
std=np.std(boston.data, axis=0)
boston_X=(boston.data - mean)/std
boston_y=boston.target

In [5]:
#making algorism
class admm_for_lasso(BaseEstimator, RegressorMixin):
    
    #initial setting
    def __init__(self, lambda_=1.0, rho=1.0, max_repeat=1000, fit_intercept=True):
        self.lambda_=lambda_ # regularized coefficient
        self.rho=rho # penalty coefficient
        self.max_repeat=max_repeat # the number of repeats
        self.fit_intercept=fit_intercept #whether calculating the intercept or not
        self.coef_=None # variable for coefficients
        self.intercept_=0.0 # variable for intercept
    
    # preparing for thresholding
    def _soft_thresholding_operator(self, x):
        alpha=self.lambda_ / self.rho
        y = np.zeros(x.shape)
        
        y[x>=alpha]= x[x>=alpha] - alpha
        y[x<=alpha]= x[x<=alpha] + alpha
        y[abs(x)<=alpha]= 0.0
        
        return y
    
    # fitting to model
    def fit(self, feature, target):
        samples=feature.shape[0]
        columns=feature.shape[1]
        
        # initial values
        x=np.dot(feature.T, target)/samples
        z=x.copy()
        y=np.zeros(len(x))
        
        inv_matrix = np.linalg.inv(np.dot(feature.T, feature)/samples + self.rho*np.identity(columns))
        
        #　calculating for Regularization
        for iteration in range(self.max_repeat):
            x=np.dot(inv_matrix, np.dot(feature.T, target)/samples + self.rho*z - y)
            z=self._soft_thresholding_operator(x + y/self.rho)
            y+=self.rho*(x - z)
            
        # making coefficients and intercept
        self.coef_=x
        if self.fit_intercept:
            self.intercept_=np.sum(target - np.dot(feature[:,1:], np.zeros(columns)[1:]))/samples
        
        return self
        
    # calculating for prediction
    def predict(self, X):
        y=np.dot(X, self.coef_) + self.intercept_
        return y

In [6]:
# format setting
np.set_printoptions(formatter={'float': '{: 0.10f}'.format})
pd.options.display.float_format = '{: 0.10f}'.format

In [7]:
#creating model with ADMM algorism
model1=admm_for_lasso(lambda_=1.0,rho=1.0,max_repeat=1000, fit_intercept=True)
model1.fit(boston_X, boston_y)

admm_for_lasso(fit_intercept=True, lambda_=1.0, max_repeat=1000, rho=1.0)

In [8]:
# results of ADMM model
print("Lasso by ADMM")
print("intercept: " + str(model1.intercept_))
print("coefficients: ")
print(model1.coef_)

Lasso by ADMM
intercept: 22.532806324110677
coefficients: 
[-0.0000000000  0.0000000000 -0.0000000000  0.0000000000  0.0000000000
  2.7131072809 -0.0000000000  0.0000000000 -0.0000000000  0.0000000000
 -1.3434986189  0.1807938799 -3.5436116588]


In [9]:
#  comparison with linear_model in sklearn
from sklearn import linear_model

model2 = linear_model.Lasso(alpha=1.0, max_iter=1000)
model2.fit(boston_X, boston_y)
print("Lasso by sickit-learn (coordinate descent)")
print("intercept: " + str(model2.intercept_))
print("coefficients: ")
print(model2.coef_)

Lasso by sickit-learn (coordinate descent)
intercept: 22.53280632411069
coefficients: 
[-0.0000000000  0.0000000000 -0.0000000000  0.0000000000 -0.0000000000
  2.7133552962 -0.0000000000 -0.0000000000 -0.0000000000 -0.0000000000
 -1.3435487980  0.1809566444 -3.5433810699]


In [10]:
# coefficients comparison
df_Lasso=pd.DataFrame()
df_Lasso["features"]=boston.feature_names
df_Lasso["by ADMM algorism"]=model1.coef_
df_Lasso["by sklearn (coordinate descent)"]=model2.coef_
df_Lasso=df_Lasso.set_index("features")
print("coefficients for housing price in Boston")
print(df_Lasso)

coefficients for housing price in Boston
          by ADMM algorism  by sklearn (coordinate descent)
features                                                   
CRIM         -0.0000000000                    -0.0000000000
ZN            0.0000000000                     0.0000000000
INDUS        -0.0000000000                    -0.0000000000
CHAS          0.0000000000                     0.0000000000
NOX           0.0000000000                    -0.0000000000
RM            2.7131072809                     2.7133552962
AGE          -0.0000000000                    -0.0000000000
DIS           0.0000000000                    -0.0000000000
RAD          -0.0000000000                    -0.0000000000
TAX           0.0000000000                    -0.0000000000
PTRATIO      -1.3434986189                    -1.3435487980
B             0.1807938799                     0.1809566444
LSTAT        -3.5436116588                    -3.5433810699
